In [4]:
#sdk package
import fnv
import fnv.reduce
import fnv.file 
#
import numpy as np
import seaborn as sns
import datetime as dt
import time
import pandas as pd
from tkinter import filedialog
from tkinter import *
import os
import matplotlib.pyplot as plt
import glob
import h5py
import gc
import shlex
import pipes
from subprocess import check_call

gc.collect() #memory management. limited utility?



...........s...........................................x...........................s.....s......................................x....................................sC:\Users\cloftus\Anaconda3\lib\site-packages\h5py\_hl\files.py:258: H5pyDeprecationWarning: File.fid has been deprecated. Use File.id instead.
  "Use File.id instead.", H5pyDeprecationWarning)
...s......ss.ss...............................................................................ss...................ssssss.....................................................................x....x.........................x......x..................................................ssss...................s........................................
----------------------------------------------------------------------
Ran 509 tests in 1.800s

OK (skipped=22, expected failures=6)


<unittest.runner.TextTestResult run=509 errors=0 failures=0>

In [5]:
options = np.array([000000, 200000, 400000, 600000, 800000, 1000000]) #milliseconds for rounding

def round_to_fifth(time_): #round timestamp to fifths of second. Useful for synchronizing videos
    
    new = options[ abs(time_.microsecond - options).argmin() ]
    
    if new != 1000000:
        new_time = time_.replace(microsecond = new)
    else:
        temp_time = time_ + dt.timedelta(seconds = 1)
        new_time = temp_time.replace(microsecond = 000000)
    
    return(new_time)

def time_correct( _timestamp_ ): #not functional. used to correct times from non-synchronized sources. Requires setting offset values
    if drive == 'D:/thermal_data/':
        _timestamp_ = _timestamp_ - time_offset_D
    if drive == 'F:/':
        _timestamp_ = _timestamp_ - time_offset_F
        
    return(_timestamp_)

In [18]:
_date_list_ = ['2019-08-19', '2019-08-20', '2019-08-21', '2019-07-31'] #strings to loop over for video loading

In [19]:
## writing all frames from a given thermal recording, separated into minute bins. Timestamps unrounded

zoom = 5 #frame size for matplotlib

start_frame = 0 #can manually modify to restart processing midway through video

vid_length = 1*60*60 #breaks output into 1 hour files

drive = 'E:/' #location of input files

start_timestamp = _date_list_[ 0 ] #rewrite 0 as index variable to loop over (or manually edit for each file)

write_to = 'D:/thermal_vids/' + start_timestamp #just dates, not timestamps

#write_to = 'L:/analysis/annotation_vids/' + start_timestamp

start_dt = dt.datetime.strptime(start_timestamp, '%Y-%m-%d') #interpret date string as datetime class

curr_date = f'{start_dt.year:04}' + f'{start_dt.month:02}' + f'{start_dt.day:02}' #remove hyphens (for file reading purposes)

os.chdir( drive + curr_date )

file_name = glob.glob('*.seq') #regular expression to find all files with seq format (only 1 file per folder)

im = fnv.file.ImagerFile(file_name[0]) #replace 0 with looping index for all files in file_name

im.get_frame(0) #first frame

print( 'got_frame')

day1 = int(im.frame_info[0]['value'][:3]) #0 or value refers to timestamp, first three characters of timestamp (number of days from Jan 1)

start = f'{start_dt.year:04}-' + f'{start_dt.month:02}-' + f'{start_dt.day:02} ' + im.frame_info[0]['value'][4:] #create a POSIX timestamp string

start_dt = dt.datetime.strptime(start, '%Y-%m-%d %H:%M:%S.%f') #create datetime class object

og_start_dt = start_dt #holds the starting day for when files cross midnight


#create directory for writing if it doesn't exist and then move to it
if not os.path.isdir( write_to + '/' ):
    
    os.makedirs ( write_to + '/')

os.chdir( write_to + '/')


final = np.array(im.final, copy=False).reshape((im.height, im.width)) #converts frame into matrix. im.final gets frame in appropriate(?) format
    
#numbers for clean printing
width = final.shape[1] / (100 / zoom)
    
height = final.shape[0] / (100 / zoom)

fig = plt.figure( figsize = ( width, height ) )
#end printing (determines the quality/size of final video)

if start_frame > 0:
    
    counter = 0 #helps to skip first part of video that has already been processed
    
else:
    
    counter = 1

#loop to process all frames in file
for i in range(start_frame, im.num_frames):
    
    if im.get_frame( i ) == False: #infroms you if file connection is broken (otherwise hard to know)
        blaasdfadff

    im.get_frame(i)

    #print( 'get_frame: ' , time.time() - start__ )

    #start__ = time.time()
    
    day2 = int(im.frame_info[0]['value'][:3]) #for keeping track of midnight date transitions

    day_diff = day2 - day1

    timestamp = f'{og_start_dt.year:04}-' + f'{og_start_dt.month:02}-' + f'{(og_start_dt.day+day_diff):02} ' + im.frame_info[0]['value'][4:]

    timestamp = dt.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S.%f')
    
    #print( 'timestamp_info: ' , time.time() - start__ )

    #start__t = time.time()
    
    
    #start new file after 1 hour. Assumes files have been written during earlier iterations of the loop
    if timestamp > ( start_dt + dt.timedelta( seconds = vid_length ) ): #if more then 1 hour
        
        if not counter == 0: 
           
            str_name = start_dt.strftime("%Y%m%d_%H%M%S%f")

            # make a text_file of all the tiffs in the folder
            text_file = open(str_name + '.txt', 'w')

            files = sorted( os.listdir() )
            
#write the name of all tiff files in folder to text file
            for file in files:

                if 'tiff' in file:

                    n = text_file.write('file ' + "'" + file + "'" + '\n')

            text_file.close()

            ## command line script for turning the series of pictures produced in the script below to movies for upload

            txt_name = str_name + '.txt'
            vid_name = str_name + '.mp4'

            command = 'ffmpeg -f concat -r 60 -i ' + pipes.quote( txt_name ) + ' -c:v libx264 -crf 15 -pix_fmt yuv420p -vf pad=ceil(iw/2)*2:ceil(ih/2)*2 ' + pipes.quote(vid_name) #ggogle ffmpg for instructions on arguments (pad is important for unknown reasons)
            check_call(shlex.split(command)) #sends to console/terminal

        
        to_delete = glob.glob('*.tiff') #removes all tiffs once video is created (memory management)
        
        for item in to_delete:
            os.remove( item )
        
        start_dt = timestamp #when saving video, set new start time for managing next chunk
        
        counter = 1
        #os.makedirs ( write_to + '/' + start_dt.strftime("%Y%m%d_%H%M%S%f") )
        
        #os.chdir( write_to + '/' + start_dt.strftime("%Y%m%d_%H%M%S%f") )
     
    
    #print( 'if_statement: ' , time.time() - start__t )

    #start__ = time.time()
        
    final = np.array(im.final, copy=False).reshape( ( im.height, im.width ) )

    final1 = final - final.mean() #centers values, might help with heatmap scales?
    
    #print( 'matrix: ' , time.time() - start__ )

    #start__ = time.time()

    sns.heatmap(final1, vmin = -np.std(final1), vmax = final1.max(), xticklabels = False, yticklabels = False, cbar = False) #sets color scale. Anything less than 1 sd below mean temp is black. Comes from matplotlib
#isotherm function here?

#labels video files with timestamp
    if final1.shape[1] > 1000:

        plt.xlabel(  timestamp.strftime("%Y-%m-%d %H:%M:%S.%f") , fontsize = 55*zoom / 5 ) #make label approriate size for video resolution (varies by camera)

    else:

        plt.xlabel(  timestamp.strftime("%Y-%m-%d %H:%M:%S.%f") , fontsize = 25*zoom / 5 )

    
    #print( 'plotting: ' , time.time() - start__ )

    #start__ = time.time()

    plt.savefig( f'{i:06}_' + timestamp.strftime("%Y%m%d_%H%M%S%f") + '.tiff' , bbox_inches = 'tight') #saves tiff file. counter at start sequences helps sort files (important for ffmpg)
    
    #print( 'save_plot: ' , time.time() - start__ )

    #start__ = time.time()
    
    fig.clear() #memory management
    
    del day2, day_diff, final, final1 #memory management
    
    gc.collect()#memory management
    
    #print( 'clean_up: ' , time.time() - start__ )
    
    #print( 'total_iter: ' , time.time() - start__t )


    
    #h = hpy()
    #print(h.heap())


got_frame


<Figure size 3686.4x2764.8 with 0 Axes>

In [20]:
#Create video file for final segment
str_name = start_dt.strftime("%Y%m%d_%H%M%S%f")

# make a text_file of all the tiffs in the folder
text_file = open(str_name + '.txt', 'w')

files = sorted( os.listdir() )

for file in files:

    
    if 'tiff' in file:

        n = text_file.write('file ' + "'" + file + "'" + '\n')

text_file.close()

## command line script for turning the series of pictures produced in the script below to movies for upload

txt_name = str_name + '.txt'
vid_name = str_name + '.mp4'

command = 'ffmpeg -f concat -r 60 -i ' + pipes.quote( txt_name ) + ' -c:v libx264 -crf 15 -pix_fmt yuv420p -vf pad=ceil(iw/2)*2:ceil(ih/2)*2 ' + pipes.quote(vid_name)
check_call(shlex.split(command))


#to_delete = glob.glob('*.tiff')

#for item in to_delete:
#    os.remove( item )

0

In [21]:
to_delete = glob.glob('*.tiff')

for item in to_delete:
    os.remove( item )